In [1]:
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
# from ucimlrepo import fetch_ucirepo 

# # fetch dataset 
# optical_recognition_of_handwritten_digits = fetch_ucirepo(id=80) 
  
# # data (as pandas dataframes) 
# X = optical_recognition_of_handwritten_digits.data.features 
# y = optical_recognition_of_handwritten_digits.data.targets 
  
# # metadata 
# print(optical_recognition_of_handwritten_digits.metadata) 

# # variable information 
# print(optical_recognition_of_handwritten_digits.variables) 

In [4]:
# y= y.to_numpy()
# y =y.ravel()

In [3]:
import numpy as np
import random

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [7]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# rf_classifier.fit(X_train, y_train)

# y_pred = rf_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [8]:
# svm_classifier = SVC(kernel='linear', C=1.0)

# svm_classifier.fit(X_train, y_train)

# y_pred = svm_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [9]:
# xgb_classifier = xgb.XGBClassifier()
# xgb_classifier.fit(X_train, y_train)

# y_pred = xgb_classifier.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [11]:
def cal_pop_fitness(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:,columns]
        rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
        rf_classifier.fit(X_train_ind, y_train)

        y_pred = rf_classifier.predict(X_test.iloc[:,columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_svm(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        svm_classifier = SVC(kernel='linear', C=1.0)
        svm_classifier.fit(X_train_ind, y_train)

        y_pred = svm_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def fit_xgboost(pop, X_train, y_train, X_test, y_test):
    fitness = []
    for i in range(len(pop)):
        columns = np.unique(pop[i])
        
        X_train_ind = X_train.iloc[:, columns]
        xgb_classifier = xgb.XGBClassifier()
        xgb_classifier.fit(X_train_ind, y_train)

        y_pred = xgb_classifier.predict(X_test.iloc[:, columns])

        accuracy = accuracy_score(y_test, y_pred)

        fitness.append(accuracy)

    return fitness

def select_mating_pool(pop, fitness, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = np.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = np.where(fitness == np.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -1
    return parents

def crossover(parents, offspring_size):
    offspring = np.empty(offspring_size)

    for k in range(offspring_size[0]):
        # Always perform one-point crossover.
        crossover_point = np.random.randint(1, parents.shape[1])
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        offspring[k, :crossover_point] = parents[parent1_idx, :crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]

    return offspring

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    for idx in range(offspring_crossover.shape[0]):
        # The random value to be added to the gene.
        random_value = np.random.choice([-1, 0, 1], 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

In [12]:
col_len = 28

sol_per_pop = 16
num_parents_mating = 8

pop_size = (sol_per_pop,col_len)

max_value = 783 
std_dev = 150
mean = max_value / 2

new_population = np.random.normal(mean, std_dev, pop_size).astype(int)

new_population[new_population > max_value] = max_value
new_population[new_population < 0] = 0

print(new_population)

[[514 445 313 495 376 440 221 731 184 632 613 209 489 308 261 106 635 623
  431 287 277 509 645 476 205 506   5 586]
 [542 436 541 527 398 171 560 379 575 292 173 475 547 338 166 577 402 182
  413 472 566 623 783 373 383 460 415 471]
 [387 596  82 501 507 267 553 529 316 297 178 348 319 549 466 538 328 335
  319 380 546 268 783 438 434 737 366 597]
 [329 649 434  96 145 525 445 437 773 486 385 546 350 475 461 331 310 513
  529 304   7 539 603 321 194 421 637 399]
 [359 442 156 439 369 533 308 399 385 327 301 393 456 520 596 314 415 457
  267 283 415 134 742 379 315 461 528 356]
 [774 328 371 561 125 425 288 400 457 379 319 492 411 289 247 325 352 614
  409 385 572 554 462 401 287 347 397 372]
 [348 470 507 469 334 168 239   0 426   0 174 423 437 422 489 201 109 145
  496 233 359 558 284 432 334 214 396 548]
 [306 386 571 404 465 244 375 232 358 290 458 295 615 320 573 396  60 366
  458 642 761 602 135 365 210 361 355 269]
 [446 188 615 617 300 385 301 460 424 363 584 284 358 432 407 29

In [13]:
y=y.astype('int64')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = X_train[:10000]
X_test = X_test[:3000]
y_train = y_train[:10000]
y_test = y_test[:3000]

In [15]:
num_generations = 10
for generation in range(num_generations):
    # Measing the fitness of each chromosome in the population.
    fitness = cal_pop_fitness(new_population, X_train,y_train,X_test,y_test)

    print(fitness)

    # Selecting the best parents in the population for mating.
    parents = select_mating_pool(new_population, fitness, 
                                      num_parents_mating)
    
    # print(parents)

    # Generating next generation using crossover.
    offspring_crossover = crossover(parents,
                                       offspring_size=(pop_size[0]-parents.shape[0], col_len))


    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(offspring_crossover)

    # print(offspring_mutation)

    # Creating the new population based on the parents and offspring.
    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation  # should be offspring_mutation

    # The best result in the current iteration.
    if generation%10 ==0:
        print("Generation : ", generation)
        # print(offspring_crossover)
        print()

[0.7606666666666667, 0.7906666666666666, 0.8053333333333333, 0.7316666666666667, 0.7493333333333333, 0.798, 0.681, 0.7986666666666666, 0.7153333333333334, 0.7423333333333333, 0.7133333333333334, 0.8116666666666666, 0.8223333333333334, 0.7373333333333333, 0.758, 0.7416666666666667]
Generation :  0

[0.8223333333333334, 0.8116666666666666, 0.8053333333333333, 0.7986666666666666, 0.798, 0.7906666666666666, 0.7606666666666667, 0.758, 0.8216666666666667, 0.813, 0.8236666666666667, 0.8073333333333333, 0.792, 0.7166666666666667, 0.7883333333333333, 0.7043333333333334]
[0.8236666666666667, 0.8223333333333334, 0.8216666666666667, 0.813, 0.8116666666666666, 0.8073333333333333, 0.8053333333333333, 0.7986666666666666, 0.8026666666666666, 0.8363333333333334, 0.8116666666666666, 0.815, 0.8053333333333333, 0.8113333333333334, 0.8206666666666667, 0.8003333333333333]
[0.8363333333333334, 0.8236666666666667, 0.8223333333333334, 0.8216666666666667, 0.8206666666666667, 0.815, 0.813, 0.8116666666666666, 0.

In [16]:
# Getting the best solution after iterating finishing all generations.
#At first, the fitness is calculated for each solution in the final generation.
fitness = cal_pop_fitness(new_population, X_train, y_train, X_test,y_test)
# Then return the index of that solution corresponding to the best fitness.
best_match_idx = np.where(fitness == np.max(fitness))

# print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx[0][0]])

Best solution fitness :  0.8566666666666667


In [17]:
print("Best solution : ", new_population[best_match_idx, :])

Best solution :  [[[443 241  12 501 505 267 553 529 316 297 178 348 319 549 466 538 486
   267 658 431 462 273 158 655 151 464 603 625]]]


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9667619047619047


In [18]:
feat = new_population[best_match_idx, :]

In [19]:
feat[0][0]

array([443, 241,  12, 501, 505, 267, 553, 529, 316, 297, 178, 348, 319,
       549, 466, 538, 486, 267, 658, 431, 462, 273, 158, 655, 151, 464,
       603, 625])

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,feat[0][0]], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,feat[0][0]])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8838571428571429


In [21]:
new_population = np.random.randint(0, 783, size=28)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

rf_classifier.fit(X_train.iloc[:,new_population], y_train)

y_pred = rf_classifier.predict(X_test.iloc[:,new_population])
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7365714285714285


In [22]:
new_population

array([395, 475, 636, 353, 577, 507, 244, 378, 711, 368, 242, 221, 458,
       517, 273, 643,  53, 141,  44, 479,  21, 582,  70, 305, 526, 178,
       463, 747])

In [1]:
0.9505714285714286/0.96676190476190470 * 100

98.32528814895085